## 구글드라이브 연동

In [459]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 파일 불러오기

In [460]:
import pandas as pd

mons = pd.read_csv("/content/drive/MyDrive/세미 프로젝트/0914_역별월별비율_15_22.csv")
days = pd.read_csv("/content/drive/MyDrive/세미 프로젝트/0914_역별요일비율(결측치해결)_인천7빼고.csv")

In [461]:
# 엑셀 파일에서 모든 시트를 읽어서 딕셔너리 형태로 저장
excel_file = pd.ExcelFile('/content/drive/MyDrive/세미 프로젝트/2011년 혼잡도.xls')
sheet_names = excel_file.sheet_names  # 시트 이름 목록
sheet_names

['평일 상선', '평일 하선', '토요일 상선', '토요일 하선', '일요일 상선', '일요일 하선']

In [462]:
# 모든 시트의 데이터를 저장할 빈 리스트를 생성
all_dfs = []

for sheet_name in sheet_names:
    # 필요한 행부터 읽기 시작
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=2)

    # 불필요한 열 삭제
    df.dropna(axis=1, how='all', inplace=True)

    # 요일구분 컬럼 추가
    day_of_week = sheet_name.split()[-2]  # 시트 이름에서 요일 정보 추출
    df['요일구분'] = day_of_week

    # 상하구분 컬럼 추가 (상선 또는 하선)
    direction = '상선' if '상선' in sheet_name else '하선'
    df['상하구분'] = direction

    # 데이터프레임을 리스트에 추가
    all_dfs.append(df)

In [463]:
all_dfs

[      호선       역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
 0    1호선    서울역.1  상선    8   16   44   76   54    38    40  ...    34    49   
 1    NaN     시청.1  상선    8   16   39   69   55    43    45  ...    42    68   
 2    NaN       종각  상선    9   13   28   53   45    40    41  ...    45    79   
 3    NaN   종로3가.1  상선    9   13   31   57   49    45    47  ...    56    97   
 4    NaN     종로5가  상선    8   12   26   46   43    40    42  ...    58   102   
 ..   ...      ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
 118  NaN     이촌.4  상선   15   23   44   77   44    35    37  ...    39    38   
 119  NaN       동작  상선   15   25   48   73   46    31    40  ...    39    38   
 120  NaN  총신대입구.4  상선   16   27   50   79   42    32    40  ...    37    38   
 121  NaN     사당.4  상선   20   25   51   77   45    33    40  ...    39    40   
 122  NaN      남태령  상선   21   36  107  135   93    59    66  ...    60    74   
 
      19시대  20시대  21시대  22시대  23시대  24

In [464]:
# 리스트에 있는 모든 데이터프레임을 합치기
df_14 = pd.concat(all_dfs, ignore_index=True)

In [465]:
df_14.info

<bound method DataFrame.info of       호선       역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
0    1호선    서울역.1  상선    8   16   44   76   54    38    40  ...    34    49   
1    NaN     시청.1  상선    8   16   39   69   55    43    45  ...    42    68   
2    NaN       종각  상선    9   13   28   53   45    40    41  ...    45    79   
3    NaN   종로3가.1  상선    9   13   31   57   49    45    47  ...    56    97   
4    NaN     종로5가  상선    8   12   26   46   43    40    42  ...    58   102   
..   ...      ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
733  NaN     이촌.4  하선   27    9    7   12   23    25    24  ...    40    35   
734  NaN       동작  하선    0   11    7   12   24    26    25  ...    40    36   
735  NaN  총신대입구.4  하선    0   12    8   14   27    31    28  ...    40    36   
736  NaN     사당.4  하선   11   12    8   17   30    39    29  ...    34    30   
737  NaN      남태령  하선   11   24   17   26   50    78    55  ...    69    64   

     19시대  20시대  21

In [466]:
# 시간대 컬럼 이름 변경
new_columns = {
    '5시대': '05시~06시', '6시대': '06시~07시', '7시대': '07시~08시', '8시대': '08시~09시', '9시대': '09시~10시',
    '10시대': '10시~11시', '11시대': '11시~12시', '12시대': '12시~13시', '13시대': '13시~14시', '14시대': '14시~15시',
    '15시대': '15시~16시', '16시대': '16시~17시', '17시대': '17시~18시', '18시대': '18시~19시', '19시대': '19시~20시',
    '20시대': '20시~21시', '21시대': '21시~22시', '22시대': '22시~23시', '23시대': '23시~24시', '24시대': '24시~01시'
}

df_14.rename(columns=new_columns, inplace=True)

In [467]:
# 컬럼 순서 변경
new_column_order = ['요일구분', '호선', '역명', '상하구분', '05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시', '10시~11시', '11시~12시',
                    '12시~13시', '13시~14시', '14시~15시', '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시', '20시~21시',
                    '21시~22시', '22시~23시', '23시~24시', '24시~01시']

In [468]:
df_14 = df_14[new_column_order]

In [469]:
df_14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   요일구분     738 non-null    object
 1   호선       24 non-null     object
 2   역명       738 non-null    object
 3   상하구분     738 non-null    object
 4   05시~06시  738 non-null    int64 
 5   06시~07시  738 non-null    int64 
 6   07시~08시  738 non-null    int64 
 7   08시~09시  738 non-null    int64 
 8   09시~10시  738 non-null    int64 
 9   10시~11시  738 non-null    int64 
 10  11시~12시  738 non-null    int64 
 11  12시~13시  738 non-null    int64 
 12  13시~14시  738 non-null    int64 
 13  14시~15시  738 non-null    int64 
 14  15시~16시  738 non-null    int64 
 15  16시~17시  738 non-null    int64 
 16  17시~18시  738 non-null    int64 
 17  18시~19시  738 non-null    int64 
 18  19시~20시  738 non-null    int64 
 19  20시~21시  738 non-null    int64 
 20  21시~22시  738 non-null    int64 
 21  22시~23시  738 non-null    int64 
 22  23

### 결측값 처리 (호선명 채우기)

In [470]:
# 비어있는 호선 값 채우기
for index, row in df_14.iterrows():
    if pd.isnull(row['호선']):  # 호선 값이 NULL인 경우
        df_14.at[index, '호선'] = df_14.at[index - 1, '호선']  # 직전 행의 호선 값 복사

In [471]:
df_14.head()

,요일구분,호선,역명,상하구분,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~01시
0,평일,1호선,서울역.1,상선,8,16,44,76,54,38,...,29,30,34,49,36,26,21,21,12,4
1,평일,1호선,시청.1,상선,8,16,39,69,55,43,...,33,39,42,68,49,38,31,31,20,5
2,평일,1호선,종각,상선,9,13,28,53,45,40,...,36,44,45,79,57,46,41,42,25,7
3,평일,1호선,종로3가.1,상선,9,13,31,57,49,45,...,43,55,56,97,71,57,49,53,31,8
4,평일,1호선,종로5가,상선,8,12,26,46,43,40,...,42,56,58,102,75,59,51,56,32,8


### df_14 데이터 형식 수정

In [472]:
# 원본 데이터프레임의 복사본 생성
df_14 = df_14.copy()

# '호선' 컬럼을 문자열로 변환하여 수정
df_14['호선'] = df_14['호선'].astype(str)

# '호선' 컬럼에서 숫자를 추출하고 정수로 변환하여 수정
df_14['호선'] = df_14['호선'].str.extract('(\d+)').fillna(0).astype(int)

# 시간대 실수형으로 형식 바꾸기
time_columns = ['05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시',
                '10시~11시', '11시~12시', '12시~13시', '13시~14시', '14시~15시',
                '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시',
                '20시~21시', '21시~22시', '22시~23시', '23시~24시', '24시~01시']

for col in time_columns:
    df_14[col] = df_14[col].astype('float64')

In [473]:
# '신천역'을 '잠실새내역'으로 변경
df_14['역명'] = df_14['역명'].replace('신천', '잠실새내')

In [474]:
# '()'와 '()'안의 글자를 모두 제거
df_14['역명'] = df_14['역명'].str.replace(r'\([^)]*\)', '', regex=True)

In [475]:
# '역명' 컬럼에서 '.숫자' 부분을 제거합니다.
df_14['역명'] = df_14['역명'].str.replace(r'\.\d+', '', regex=True)

In [476]:
df_14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   요일구분     738 non-null    object 
 1   호선       738 non-null    int64  
 2   역명       738 non-null    object 
 3   상하구분     738 non-null    object 
 4   05시~06시  738 non-null    float64
 5   06시~07시  738 non-null    float64
 6   07시~08시  738 non-null    float64
 7   08시~09시  738 non-null    float64
 8   09시~10시  738 non-null    float64
 9   10시~11시  738 non-null    float64
 10  11시~12시  738 non-null    float64
 11  12시~13시  738 non-null    float64
 12  13시~14시  738 non-null    float64
 13  14시~15시  738 non-null    float64
 14  15시~16시  738 non-null    float64
 15  16시~17시  738 non-null    float64
 16  17시~18시  738 non-null    float64
 17  18시~19시  738 non-null    float64
 18  19시~20시  738 non-null    float64
 19  20시~21시  738 non-null    float64
 20  21시~22시  738 non-null    float64
 21  22시~23시  738 non

In [477]:
df_14.isna().sum()

요일구분       0
호선         0
역명         0
상하구분       0
05시~06시    0
06시~07시    0
07시~08시    0
08시~09시    0
09시~10시    0
10시~11시    0
11시~12시    0
12시~13시    0
13시~14시    0
14시~15시    0
15시~16시    0
16시~17시    0
17시~18시    0
18시~19시    0
19시~20시    0
20시~21시    0
21시~22시    0
22시~23시    0
23시~24시    0
24시~01시    0
dtype: int64

### 혼잡도 구하기

In [478]:
melt_14 = pd.melt(df_14, id_vars=['요일구분', '호선', '역명', '상하구분'], var_name='시간대', value_name='값')
melt_14

,요일구분,호선,역명,상하구분,시간대,값
0,평일,1,서울역,상선,05시~06시,8.0
1,평일,1,시청,상선,05시~06시,8.0
2,평일,1,종각,상선,05시~06시,9.0
3,평일,1,종로3가,상선,05시~06시,9.0
4,평일,1,종로5가,상선,05시~06시,8.0
...,...,...,...,...,...,...
14755,일요일,4,이촌,하선,24시~01시,0.0
14756,일요일,4,동작,하선,24시~01시,0.0
14757,일요일,4,총신대입구,하선,24시~01시,0.0
14758,일요일,4,사당,하선,24시~01시,4.0


In [479]:
melt_14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    14760 non-null  object 
 1   호선      14760 non-null  int64  
 2   역명      14760 non-null  object 
 3   상하구분    14760 non-null  object 
 4   시간대     14760 non-null  object 
 5   값       14760 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 692.0+ KB


In [480]:
mons.rename(columns={'호선명': '호선', '지하철역': '역명'}, inplace=True)

In [481]:
mons = mons[(mons['호선'] == '1호선') | (mons['호선'] == '2호선') | (mons['호선'] == '3호선') | (mons['호선'] == '4호선')]

In [482]:
mons.호선.value_counts

<bound method IndexOpsMixin.value_counts of 0       1호선
1       1호선
2       1호선
3       1호선
4       1호선
       ... 
1447    4호선
1448    4호선
1449    4호선
1450    4호선
1451    4호선
Name: 호선, Length: 1452, dtype: object>

In [483]:
mons = mons.replace({'호선' : '1호선'}, '1')
mons = mons.replace({'호선' : '2호선'}, '2')
mons = mons.replace({'호선' : '3호선'}, '3')
mons = mons.replace({'호선' : '4호선'}, '4')

In [484]:
mons

,호선,역명,사용월,비율
0,1,동대문,1,0.954919
1,1,동대문,2,0.882286
2,1,동대문,3,1.052501
3,1,동대문,4,1.032322
4,1,동대문,5,1.080913
...,...,...,...,...
1447,4,회현,8,0.949531
1448,4,회현,9,0.942649
1449,4,회현,10,1.049186
1450,4,회현,11,1.034597


In [485]:
melt_14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    14760 non-null  object 
 1   호선      14760 non-null  int64  
 2   역명      14760 non-null  object 
 3   상하구분    14760 non-null  object 
 4   시간대     14760 non-null  object 
 5   값       14760 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 692.0+ KB


In [486]:
melt_14['호선'] = melt_14['호선'].astype(str)
merged_df = melt_14.merge(mons, on=['호선', '역명'], how='inner')
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율
0,평일,1,서울역,상선,05시~06시,8.0,1,1.002597
1,평일,1,서울역,상선,05시~06시,8.0,2,0.919665
2,평일,1,서울역,상선,05시~06시,8.0,3,1.014560
3,평일,1,서울역,상선,05시~06시,8.0,4,0.990673
4,평일,1,서울역,상선,05시~06시,8.0,5,1.034015
...,...,...,...,...,...,...,...,...
164155,일요일,4,남태령,하선,24시~01시,11.0,8,1.193308
164156,일요일,4,남태령,하선,24시~01시,11.0,9,1.096531
164157,일요일,4,남태령,하선,24시~01시,11.0,10,0.997312
164158,일요일,4,남태령,하선,24시~01시,11.0,11,1.014522


In [487]:
merged_df['월혼잡도'] = merged_df['값'] * merged_df['비율']
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,평일,1,서울역,상선,05시~06시,8.0,1,1.002597,8.020773
1,평일,1,서울역,상선,05시~06시,8.0,2,0.919665,7.357318
2,평일,1,서울역,상선,05시~06시,8.0,3,1.014560,8.116482
3,평일,1,서울역,상선,05시~06시,8.0,4,0.990673,7.925385
4,평일,1,서울역,상선,05시~06시,8.0,5,1.034015,8.272120
...,...,...,...,...,...,...,...,...,...
164155,일요일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386
164156,일요일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844
164157,일요일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433
164158,일요일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747


In [488]:
merged_df.역명.value_counts()

서울역        2880
시청         2880
교대         2880
신도림        2880
성수         2880
           ... 
구로디지털단지    1440
신대방        1440
신림         1440
봉천         1440
남태령        1440
Name: 역명, Length: 103, dtype: int64

In [489]:
days.head()

,호선,역명,요일,비율
0,1,동대문,금,1.061534
1,1,동대문,목,1.020850
2,1,동대문,수,1.026641
3,1,동대문,월,1.005251
4,1,동대문,일,0.873151


In [490]:
days.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1924 entries, 0 to 1923
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      1924 non-null   int64  
 1   역명      1924 non-null   object 
 2   요일      1924 non-null   object 
 3   비율      1924 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 60.2+ KB


In [491]:
days.rename(columns={'요일': '요일구분'}, inplace=True)
days = days[(days['호선'] == 1) | (days['호선'] == 2) | (days['호선'] == 3) | (days['호선'] == 4)]
days

,호선,역명,요일구분,비율
0,1,동대문,금,1.061534
1,1,동대문,목,1.020850
2,1,동대문,수,1.026641
3,1,동대문,월,1.005251
4,1,동대문,일,0.873151
...,...,...,...,...
835,4,회현,수,1.138622
836,4,회현,월,1.091271
837,4,회현,일,0.453112
838,4,회현,토,0.888779


In [492]:
merged_df = merged_df.replace({'요일구분' : '일요일'}, '일')
merged_df = merged_df.replace({'요일구분' : '토요일'}, '토')
merged_df.요일구분.value_counts()

평일    54720
토     54720
일     54720
Name: 요일구분, dtype: int64

In [493]:
weekday = merged_df[merged_df['요일구분'] == '평일'].copy()
# weekday
weeks = ['월', '화', '수', '목', '금']
result_df = pd.concat([weekday.assign(요일구분=요일) for 요일 in weeks], ignore_index=True)
result_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,월,1,서울역,상선,05시~06시,8.0,1,1.002597,8.020773
1,월,1,서울역,상선,05시~06시,8.0,2,0.919665,7.357318
2,월,1,서울역,상선,05시~06시,8.0,3,1.014560,8.116482
3,월,1,서울역,상선,05시~06시,8.0,4,0.990673,7.925385
4,월,1,서울역,상선,05시~06시,8.0,5,1.034015,8.272120
...,...,...,...,...,...,...,...,...,...
273595,금,4,남태령,하선,24시~01시,19.0,8,1.193308,22.672848
273596,금,4,남태령,하선,24시~01시,19.0,9,1.096531,20.834094
273597,금,4,남태령,하선,24시~01시,19.0,10,0.997312,18.948929
273598,금,4,남태령,하선,24시~01시,19.0,11,1.014522,19.275927


In [494]:
result_df.tail(30)

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
273570,금,4,남태령,하선,23시~24시,60.0,7,1.074550,64.473019
273571,금,4,남태령,하선,23시~24시,60.0,8,1.193308,71.598467
273572,금,4,남태령,하선,23시~24시,60.0,9,1.096531,65.791875
273573,금,4,남태령,하선,23시~24시,60.0,10,0.997312,59.838724
273574,금,4,남태령,하선,23시~24시,60.0,11,1.014522,60.871347
273575,금,4,남태령,하선,23시~24시,60.0,12,0.987340,59.240404
273576,금,4,남태령,상선,24시~01시,9.0,1,0.740849,6.667643
273577,금,4,남태령,상선,24시~01시,9.0,2,0.750525,6.754722
273578,금,4,남태령,상선,24시~01시,9.0,3,1.019584,9.176256
273579,금,4,남태령,상선,24시~01시,9.0,4,1.069612,9.626507


In [495]:
result_df.역명.value_counts()

서울역        4800
시청         4800
교대         4800
신도림        4800
성수         4800
           ... 
구로디지털단지    2400
신대방        2400
신림         2400
봉천         2400
남태령        2400
Name: 역명, Length: 103, dtype: int64

In [496]:
fin = pd.concat([result_df, merged_df[merged_df['요일구분'].isin(['토', '일'])]], ignore_index=True)
fin.요일구분.value_counts()

월    54720
화    54720
수    54720
목    54720
금    54720
토    54720
일    54720
Name: 요일구분, dtype: int64

In [497]:
fin['호선'] = fin['호선'].astype(int)
merged_fin = fin.merge(days, on=['호선', '역명', '요일구분'], how='inner')
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y
0,월,1,서울역,상선,05시~06시,8.0,1,1.002597,8.020773,1.055483
1,월,1,서울역,상선,05시~06시,8.0,2,0.919665,7.357318,1.055483
2,월,1,서울역,상선,05시~06시,8.0,3,1.014560,8.116482,1.055483
3,월,1,서울역,상선,05시~06시,8.0,4,0.990673,7.925385,1.055483
4,월,1,서울역,상선,05시~06시,8.0,5,1.034015,8.272120,1.055483
...,...,...,...,...,...,...,...,...,...,...
383035,일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386,0.627717
383036,일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844,0.627717
383037,일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433,0.627717
383038,일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747,0.627717


In [498]:
merged_fin.역명.value_counts()

서울역        6720
시청         6720
교대         6720
신도림        6720
성수         6720
           ... 
구로디지털단지    3360
신대방        3360
신림         3360
봉천         3360
남태령        3360
Name: 역명, Length: 103, dtype: int64

In [499]:
merged_fin['최종혼잡도'] = merged_fin['월혼잡도'] * merged_fin['비율_y']
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y,최종혼잡도
0,월,1,서울역,상선,05시~06시,8.0,1,1.002597,8.020773,1.055483,8.465786
1,월,1,서울역,상선,05시~06시,8.0,2,0.919665,7.357318,1.055483,7.765520
2,월,1,서울역,상선,05시~06시,8.0,3,1.014560,8.116482,1.055483,8.566804
3,월,1,서울역,상선,05시~06시,8.0,4,0.990673,7.925385,1.055483,8.365105
4,월,1,서울역,상선,05시~06시,8.0,5,1.034015,8.272120,1.055483,8.731078
...,...,...,...,...,...,...,...,...,...,...,...
383035,일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386,0.627717,8.239655
383036,일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844,0.627717,7.571424
383037,일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433,0.627717,6.886327
383038,일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747,0.627717,7.005162


In [503]:
merged_fin['연도'] = 2011

In [504]:
merged_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383040 entries, 0 to 383039
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   요일구분    383040 non-null  object 
 1   호선      383040 non-null  int64  
 2   역명      383040 non-null  object 
 3   상하구분    383040 non-null  object 
 4   시간대     383040 non-null  object 
 5   값       383040 non-null  float64
 6   사용월     383040 non-null  int64  
 7   비율_x    383040 non-null  float64
 8   월혼잡도    383040 non-null  float64
 9   비율_y    383040 non-null  float64
 10  최종혼잡도   383040 non-null  float64
 11  연도      383040 non-null  int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 38.0+ MB


In [505]:
merged_fin.isna().sum()

요일구분     0
호선       0
역명       0
상하구분     0
시간대      0
값        0
사용월      0
비율_x     0
월혼잡도     0
비율_y     0
최종혼잡도    0
연도       0
dtype: int64

In [506]:
merged_fin.to_csv("df_14_merged_fin(최종).csv", index=False)